# Consumindos dados do Dataset

## MONTANDO O DRIVE

Dataset utilizada do [Kaggle](https://www.kaggle.com/gpreda/covid-world-vaccination-progress) - **COVID-19 World Vaccination Progress**

Iniciamos importando as bibliotecas

In [4]:
#pip install pyodbc

In [10]:
import pandas as pd #Tratamento e tranformação de dataframes
import pyodbc #Conector do banco de dados SQL Server
import sys
import os

## CARREGANDO O DATASET

In [11]:
df = pd.read_csv('./country_vaccinations_by_manufacturer.csv')
df

,location,date,vaccine,total_vaccinations
0,Austria,2021-01-08,Johnson&Johnson,0
1,Austria,2021-01-08,Moderna,0
2,Austria,2021-01-08,Oxford/AstraZeneca,0
3,Austria,2021-01-08,Pfizer/BioNTech,31475
4,Austria,2021-01-15,Johnson&Johnson,0
...,...,...,...,...
26270,European Union,2022-01-07,Oxford/AstraZeneca,67339955
26271,European Union,2022-01-07,Pfizer/BioNTech,525007958
26272,European Union,2022-01-07,Sinopharm/Beijing,2242654
26273,European Union,2022-01-07,Sinovac,9


# EXPLORANDO OS DADOS

In [12]:
#VERIFICANDO SE TEM VALORES NULL

df.isnull().sum()

location              0
date                  0
vaccine               0
total_vaccinations    0
dtype: int64

Observamos que não existem dados nulos dentro do dataframe.

In [13]:
#CONTABILIZANDO OS VALORES DA TABELA VACCINE

df['vaccine'].value_counts()

Pfizer/BioNTech       7282
Oxford/AstraZeneca    5497
Moderna               5325
Johnson&Johnson       4620
Sinovac               1961
Sinopharm/Beijing      783
Sputnik V              439
CanSino                368
Name: vaccine, dtype: int64

Temos a conagem de cada ocorrencia das vacinas e a mais usada foi a Pfizer/BioNTech com 7282 dose aplicadas.

In [14]:
#VERIFICANDO QUANTAS LINHAS E COLUNAS EXISTEM

df.shape

(26275, 4)

## MANIPULAÇÃO E TRANSFORMAÇÃO DOS DADOS

In [15]:
# RENOMEANDO AS COLUNAS

df.columns=['Localizacao','Data','Nome_da_Vacina', 'Numero_de_Vacinados']
df

,Localizacao,Data,Nome_da_Vacina,Numero_de_Vacinados
0,Austria,2021-01-08,Johnson&Johnson,0
1,Austria,2021-01-08,Moderna,0
2,Austria,2021-01-08,Oxford/AstraZeneca,0
3,Austria,2021-01-08,Pfizer/BioNTech,31475
4,Austria,2021-01-15,Johnson&Johnson,0
...,...,...,...,...
26270,European Union,2022-01-07,Oxford/AstraZeneca,67339955
26271,European Union,2022-01-07,Pfizer/BioNTech,525007958
26272,European Union,2022-01-07,Sinopharm/Beijing,2242654
26273,European Union,2022-01-07,Sinovac,9


### Converção de dados

In [17]:
tipo_dados = {                
        'Localizacao': 'category',
        'Data': 'datetime64',
        'Nome_da_Vacina': 'category',
        'Numero_de_Vacinados': 'category',     
}
df = df.astype(tipo_dados)

In [18]:
#VERIFICANDO O TIPO DOS REGISTROS

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26275 entries, 0 to 26274
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Localizacao          26275 non-null  category      
 1   Data                 26275 non-null  datetime64[ns]
 2   Nome_da_Vacina       26275 non-null  category      
 3   Numero_de_Vacinados  26275 non-null  category      
dtypes: category(3), datetime64[ns](1)
memory usage: 1005.2 KB


## Inserção no banco de dados SQL Server

Possíveis drives que funcionam no Python
- SQL Server Native Client 11.0
- ODBC Driver 17 for SQL Server

In [12]:
#import pyodbc

In [19]:
## Inserção no banco de dados SQL Server
Server = ''
Database = ''

# Utilize o drive disponível no servidor e usuario logado no computador
conexao = pyodbc.connect('Driver={SQL Server Native Client 11.0};Server='+ Server +';Database='+ Database +';Trusted_Connection=yes;')
cursor = conexao.cursor()


In [14]:
# INSERT Dataframe into SQL Server:
for index, row in df.iterrows():
   cursor.execute("""
        INSERT INTO COVID_19 ([location],[date],[vaccine],[total_vaccinations]) 
        VALUES( ?, ?, ?, ? )""",
        row.Localizacao, row.Data, row.Nome_da_Vacina, row.Numero_de_Vacinados
    )

conexao.commit()
cursor.close()
print("Conexão ao SQl Server foi encerrada!")

Conexão ao SQl Server foi encerrada!


**FIM**